In [1]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 130 (delta 64), reused 104 (delta 39), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 3.42 MiB | 14.51 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  src  uv.lock


# Setup

In [2]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 66.7 MB/s eta 0:00:00:00:0100:01
Using CPython 3.12.12
Creating virtual environment at: .venv
Resolved 72 packages in 1ms
Prepared 66 packages in 53.92s                                           
░░░░░░░░░░░░░░░░░░░░ [0/66] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 66 packages in 6.81s                              
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + annotated-types==0.7.0
 + attrs==25.4.0
 + certifi==2025.11.12
 + charset-normalizer==3.4.4
 + filelock==3.20.0
 + frozenlist==1.8.0
 + fsspec==2025.12.0
 + hf-xet==1.2.0
 + huggingface-hub==0.36.0
 + idna==3.11

In [3]:
!gdown --folder https://drive.google.com/drive/folders/1_GMVo-YBjH0sQcQqjE8UhQMZqt30B6ux


Retrieving folder contents
Processing file 1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT data.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT
To: /kaggle/working/mlproduction/data/data.csv
100%|██████████████████████████████████████| 10.7M/10.7M [00:00<00:00, 51.0MB/s]
Download completed


In [4]:
import pandas as pd
df = pd.read_csv("data/data.csv")

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-multilingual-cased"
)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [8]:
from src.dataloaders.hallu_dataloader import TextPairDataModule
dm = TextPairDataModule(
    df=df,
    tokenizer=tokenizer,
    batch_size=16,
    max_length=256,
)
dm.setup()


In [9]:
batch = next(iter(dm.train_dataloader()))

print(batch.keys())
print(batch["input_ids"].shape)
print(batch["labels"])


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
torch.Size([16, 256])
tensor([1, 0, 1, 2, 2, 1, 1, 2, 0, 2, 2, 2, 0, 0, 1, 1])


In [11]:
import torch
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification

class TextClassifier(pl.LightningModule):
    def __init__(
        self,
        model_name: str,
        num_labels: int,
        lr: float = 2e-5,
    ) -> None:
        super().__init__()
        self.save_hyperparameters()

        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
        )

    def forward(self, **batch):
        return self.model(**batch)

    def training_step(self, batch, batch_idx):
        outputs = self(**batch)
        self.log("train_loss", outputs.loss)
        return outputs.loss

    def validation_step(self, batch, batch_idx):
        outputs = self(**batch)
        self.log("val_loss", outputs.loss)

    def configure_optimizers(self):
        return torch.optim.AdamW(
            self.parameters(),
            lr=self.hparams.lr,
        )


In [12]:
from pytorch_lightning import Trainer

model = TextClassifier(
    model_name="bert-base-multilingual-cased",
    num_labels=len(dm.label_encoder.classes_),
)

trainer = Trainer(
    max_epochs=3,
    accelerator="auto",
    devices="auto",
)

trainer.fit(model, datamodule=dm)


2025-12-16 17:20:07.729511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765905608.103362      79 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765905608.219640      79 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:527: Found 231 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 598, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 1011, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 1055, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py", line 216, in run
    self.advance()
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py", line 458, in advance
    self.epoch_loop.run(self._data_fetcher)
  File "/usr/local/lib/python3.11

TypeError: object of type 'NoneType' has no len()

In [13]:
batch = next(iter(dm.val_dataloader()))
outputs = model(**batch)

pred_ids = outputs.logits.argmax(dim=-1)

labels = dm.label_encoder.inverse_transform(
    pred_ids.cpu().numpy()
)

print(labels)


['extrinsic' 'extrinsic' 'extrinsic' 'extrinsic' 'extrinsic' 'extrinsic'
 'extrinsic' 'extrinsic' 'extrinsic' 'extrinsic' 'extrinsic' 'extrinsic'
 'extrinsic' 'extrinsic' 'extrinsic' 'extrinsic']
